# Import

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
html_df = pd.read_html('https://www.avma.org/resources-tools/literature-reviews/dog-bite-risk-and-prevention-role-breed')

In [3]:
high_risk = pd.DataFrame(html_df[0])

### Remove the zero-width space character from the column names

In [4]:
for col in high_risk.columns:
    high_risk.rename(columns={col: col.replace('\u200b', '')}, inplace=True)
high_risk.head()

,Period,Data source,N,Country,Top two breeds edentified,Ref
0,​1971,​US Dept. Health,​843,United States (VA)​,​mixed breed German Shepherd Dog,1​
1,​1971-1974,Hospital records​,50​,​South Africa,German Shepherd Dog Labrador Retriever​,​2
2,​1973-1976,US Dept. Health​,​2618,​United States (AL),German Shepherd Dog Collie​,​3
3,​1979-1982,​Health Dept. Severe attacks,16​,​United States (SC),pit bull type Saint Bernard,​21
4,1981-1983​,​US Reservations,772​,United States​,​mixed breed unspecified pedigree,19​


### Remove values inside parentheses

In [5]:
high_risk['Country'] = high_risk['Country'].str.replace(r'\s*\([^()]*\)', '', regex=True)
high_risk['Country'].unique()

array(['United States\u200b', '\u200bSouth Africa', '\u200bUnited States',
       'Canada\u200b', 'United Kingdom\u200b', '\u200bNew Zealand',
       '\u200bUnited Kingdom', '\u200bSpain', '\u200bAustralia',
       '\u200bCanada', 'New Zealand\u200b', 'Austria\u200b',
       'New Zealand', '\u200bIreland', 'Switzerland'], dtype=object)

### Replace \u200b from values any column

In [6]:
high_risk = high_risk.applymap(lambda x: x.replace('\u200b', '') if isinstance(x, str) else x)
high_risk['Country'].unique()

array(['United States', 'South Africa', 'Canada', 'United Kingdom',
       'New Zealand', 'Spain', 'Australia', 'Austria', 'Ireland',
       'Switzerland'], dtype=object)

### Drop any rows where the value is "?"

In [7]:
high_risk['N'].replace("?", np.nan, inplace=True)
high_risk = high_risk.dropna(subset=["N"])
high_risk['N'].head()

0     843
1      50
2    2618
3      16
4     772
Name: N, dtype: object

### Break out breeds that were identified as higher risk

In [8]:
# split out breed names into list
high_risk = high_risk.assign(Breed=high_risk['Top two breeds edentified'].str.split('  ')).explode('Top two breeds edentified')
high_risk = high_risk.explode('Breed').reset_index()
high_risk.head()

,index,Period,Data source,N,Country,Top two breeds edentified,Ref,Breed
0,0,1971,US Dept. Health,843,United States,mixed breed German Shepherd Dog,1,mixed breed
1,0,1971,US Dept. Health,843,United States,mixed breed German Shepherd Dog,1,German Shepherd Dog
2,1,1971-1974,Hospital records,50,South Africa,German Shepherd Dog Labrador Retriever,2,German Shepherd Dog
3,1,1971-1974,Hospital records,50,South Africa,German Shepherd Dog Labrador Retriever,2,Labrador Retriever
4,2,1973-1976,US Dept. Health,2618,United States,German Shepherd Dog Collie,3,German Shepherd Dog


In [9]:
# https://data.world/len/dog-canine-breed-size-akc
breed_df = pd.read_csv('akc_breed_info.csv')

In [10]:
# function that determines ratio and token
def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup)],
                     ['ratio', 'token'])
# create multi index to use fuzzy match
compare = pd.MultiIndex.from_product([breed_df['breed'],
                                      high_risk['Breed'].unique()]).to_series()
# Create reference to join views
breed_fuzzy = compare.apply(metrics).unstack().idxmax().unstack(0).reset_index()

In [11]:
# fix matches found by manually checking file
def update_ratio(index, ratio):
    if index.lower() in ['american bulldog','american bulldog mix','english bulldog','english bulldog mix','old english bulldog']:
        return 'Bull Dog'
    elif index.lower() in ['lhasa apso','lhasa apso mix']:
        return 'Tibetan Terrier'
    elif index.lower() in ['miniature pinscher','miniature pinscher mix']:
        return 'Dachshund'
    elif index.lower() in ['pit bull','pit bull type', 'pitt']:
        return 'American Staffordshire Terrier'
    elif index.lower() in ['alaskan malamute','alaskan malamute mix']:
        return 'Siberian Husky'
    else:
        return ratio
breed_fuzzy['ratio'] = breed_fuzzy[['index','ratio']].apply(lambda breed_fuzzy: update_ratio(breed_fuzzy['index'],breed_fuzzy['ratio']),axis=1)
breed_fuzzy = breed_fuzzy[['index','ratio']].rename(columns={'index' : 'Breed_join', 'ratio' : 'Breed_primary'})

In [12]:
# joins breed dataframe with high risk for combining all in the future
high_risk = pd.merge(high_risk, breed_fuzzy, how='left', left_on='Breed', right_on='Breed_join')

In [13]:
# removes records where they don't specify a specific breed
high_risk = high_risk[~high_risk['Breed'].isin(['"sled dog"','Mixed breed','mixed breed','unspecified pedigree', '--'])]

In [14]:
# create a final dataframe with the breed and the number of times identified
high_risk_df = high_risk.groupby('Breed_primary').agg(identify_cnt=('Breed_primary', 'count')).reset_index()

In [15]:
print(high_risk_df)

                     Breed_primary  identify_cnt
0   American Staffordshire Terrier             9
1            Black Russian Terrier             3
2                    Border Collie             2
3                     Bull Terrier             3
4                        Chow Chow             1
5           Cocker Spaniel-English             1
6                    Field Spaniel             1
7              German Shepherd Dog            22
8               Labrador Retriever             1
9                       Rottweiler             7
10                   Saint Bernard             1
11                  Siberian Husky             1
12      Staffordshire Bull Terrier             1
13          Welsh Springer Spaniel             1


&emsp; There are several ethical implications to consider when pulling data from tables on a webpage.
1. Intellectual Property Rights: Another ethical consideration is the intellectual property rights of the website owner. If the website owner has copyrighted the data on their website, using it without their permission could be a violation of their rights.
2. Data Accuracy: The accuracy and integrity of the data also need to be considered. If the data is inaccurate or incomplete, it could lead to biased or unreliable machine learning models.
3. Fair Use: Fair use is an important ethical consideration when using data from tables on a webpage. Fair use allows for the limited use of copyrighted material without permission, but only under certain circumstances, such as for educational or research purposes.

In [17]:
high_risk_df.to_csv('high_risk_clean_df.csv', index=False)